# FedHeart, Comparisson Between Clear, SecAgg-JLS and SecAgg-LOM 

### FLamby configuration - Download FedHeart

You need to download the FLamby dataset that we will use. For licensing reasons, these are not included directly in the FLamby installation.

To download the fed_heart dataset in `${FEDBIOMED_DIR}/data` (where `${FEDBIOMED_DIR}` is the base directory of Fed-BioMed): 

1. `source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment node`<br>
2. `pip install wget`<br>
3. `python ${FEDBIOMED_DIR}docs/tutorials/sec-agg/fed-heart/download_fead_hart.py --output-folder ${FEDBIOMED_DIR}/data`

In [1]:
import os
import torch 
from torch.utils.data import DataLoader
import numpy as np
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_heart_disease import FedHeartDisease
from fedbiomed.common.data import FlambyDataset, DataManager
from torch.nn.modules.loss import _Loss
import torch.nn as nn
from fedbiomed.researcher.environ import environ

In [2]:
FEDBIOMED_DIR = os.getenv('FEDBIOMED_DIR')
DATASET_TEST_PATH = f"{FEDBIOMED_DIR}/data"

## 1. Fed-BioMed

Configuring the Fed-BioMed training plan involves specifying the machine learning model, defining the loss function, and identifying the necessary dependencies. This ensures a clear and well-defined setup for the training process.

In [3]:
%load_ext tensorboard

In [5]:
class FedHeartTrainingPlan(TorchTrainingPlan):
    
    class Baseline(nn.Module):
        
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(13, 16)
            self.fc2 = nn.Linear(16, 2)
            self.act = nn.LeakyReLU()
        def forward(self, x):
            x = self.act(self.fc1(x))
            x = self.fc2(x)
            return x
        
    class BaselineLoss(_Loss):
        def __init__(self):
            super().__init__()
            self.ce = torch.nn.CrossEntropyLoss()

        def forward(self, prediction: torch.Tensor, target: torch.Tensor):
            target = torch.squeeze(target, dim=1).type(torch.long)
            return self.ce(prediction, target)
    
    def init_model(self, model_args):
        return self.Baseline()

    def init_optimizer(self, optimizer_args):
        return torch.optim.AdamW(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_heart_disease import FedHeartDisease",
                "from torch.nn.modules.loss import _Loss",
                "from fedbiomed.common.data import FlambyDataset, DataManager"]

    def training_step(self, data, target):
        logits = self.model().forward(data)
        return self.BaselineLoss().forward(logits, target)

    def training_data(self, batch_size=2):
        dataset = FlambyDataset()
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset, **train_kwargs)

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())
count_parameters(FedHeartTrainingPlan.init_model())

NameError: name 'FedHeartTrainingPlan' is not defined

In [5]:
batch_size = 8
num_updates = 10
num_rounds = 75

In [6]:
training_args = {
    'optimizer_args': {
        'lr': 5e-4,
    },
    'loader_args': {
        'batch_size': batch_size,
    },
    'num_updates': num_updates,
    'dry_run': False,
    'log_interval': 2,
    'test_ratio' : 0.0,
    'test_on_global_updates': False,
    'test_on_local_updates': False,
    'random_seed':42,
}

model_args = {}

## 2. Federated Learning Training with SecAgg

**Nodes Configuration:**
The FLamby Fed-Heart benchmark relies on 4 nodes.
For each node in the range `i in [0...3]` (`0` and `3` are both included):

1. Open a new terminal.
2. Run the command: `./scripts/fedbiomed_run node --config config-n{i}.ini dataset add`
3. Select `6) flamby`.
4. Enter the dataset name: `flamby` (optional).
5. Set tags to: `heart` (important).
6. Description: Enter `none` (optional).
7. Select `1) fed_heart_disease`.
8. Specify a center ID between 0 and 3: `{i}`.
9. Description: Enter `none` (optional).
10. Run the command: `./scripts/fedbiomed_run node --config config-n{i}.ini start`.

### Configuring Secure Aggregation

If you haven't done so already, you need to configure SecAgg in Fed-BioMed. 
A quick reminder of the basic commands is given below, but please refer to our documentation for a full explanation.

1. `${FEDBIOMED_DIR}/scripts/fedbiomed_configure_secagg researcher`
2. make sure the researcher and nodes' config files have been created (e.g. step 2 above)
3. `${FEDBIOMED_DIR}/scripts/fedbiomed_run certificate-dev-setup`


In [7]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.researcher.secagg import SecureAggregation

tags =  ['heart']

## 3. Training w/o SecAgg

In [8]:

exp_no_sec_agg = Experiment(tags=tags,
                 training_plan_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 model_args=model_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage())
exp_no_sec_agg.set_retain_full_history(True)

2024-05-25 09:05:22,246 fedbiomed INFO - Starting researcher service...

2024-05-25 09:05:22,262 fedbiomed INFO - Waiting 3s for nodes to connect...

2024-05-25 09:05:23,586 fedbiomed DEBUG - Node: NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 polling for the tasks

2024-05-25 09:05:23,587 fedbiomed DEBUG - Node: NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 polling for the tasks

2024-05-25 09:05:23,588 fedbiomed DEBUG - Node: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 polling for the tasks

...

2024-05-25 09:05:25,291 fedbiomed INFO - Node selected for training -> NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30

2024-05-25 09:05:25,292 fedbiomed INFO - Node selected for training -> NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8

2024-05-25 09:05:25,295 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0042/model_d10c6fc0-6383-44fa-979c-2a0b61506653.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


True

In [9]:
exp_no_sec_agg.run()

2024-05-25 09:05:25,307 fedbiomed INFO - Sampled nodes in round 0 ['NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4', 'NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42', 'NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30', 'NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8']

2024-05-25 09:05:25,309 fedbiomed INFO - Sending request 
					 To: NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:05:25,310 fedbiomed INFO - Sending request 
					 To: NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:05:25,310 fedbiomed INFO - Sending request 
					 To: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:05:25,311 fedbiomed INFO - Sending request 
					 To: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 
					 Request: : TRAIN
 -----------------------------------------------------------------

...

2024-05-25 09:06:01,785 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 
					 Round 75 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 0.045831 
					 ---------

2024-05-25 09:06:01,800 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 
					 Round 75 | Iteration: 8/10 (80%) | Samples: 16/80
 					 Loss: 1.205237 
					 ---------

2024-05-25 09:06:01,822 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 
					 Round 75 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 0.129506 
					 ---------

2024-05-25 09:06:01,875 fedbiomed INFO - Nodes that successfully reply in round 74 ['NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4', 'NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42', 'NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30', 'NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8']

75

In [10]:
def print_training_rounds(exp, num_rounds):
    # List the training rounds
    print("\nList the training rounds:", exp.training_replies().keys())

    # Iterate over each training round and print details for each node
    print("\nList the nodes for the last training round and their timings:")
    for rnd in range(num_rounds):
        round_data = exp.training_replies()[rnd]
        for r in round_data.values():
            print("\t- {id} :\
            \n\t\trtime_training={rtraining:.2f} seconds\
            \n\t\tptime_training={ptraining:.2f} seconds\
            \n\t\ttime_encrypt={time_encrypt:.2f} seconds\
            \n\t\tptxt_model_size={ptxt_model_size:.2f} MB\
            \n\t\tctxt_model_size={ctxt_model_size:.2f} MB\
            \n\t\trtime_total={rtotal:.2f} seconds".format(
                id=r['node_id'],
                rtraining=r['timing']['rtime_training'],
                ptraining=r['timing']['ptime_training'],
                time_encrypt=r['timing']['time_encrypt'],
                ptxt_model_size=r['communication']['ptxt_model_size'],
                ctxt_model_size=r['communication']['ctxt_model_size'],
                rtotal=r['timing']['rtime_total']
            ))
        print('\n')

In [11]:
print_training_rounds(exp_no_sec_agg, num_rounds)


List the training rounds: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74])

List the nodes for the last training round and their timings:
	- NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 :            
		rtime_training=0.08 seconds            
		ptime_training=0.15 seconds            
		time_encrypt=0.00 seconds            
		ptxt_model_size=0.00 MB            
		ctxt_model_size=0.00 MB            
		rtime_total=0.39 seconds
	- NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 :            
		rtime_training=0.16 seconds            
		ptime_training=0.34 seconds            
		time_encrypt=0.00 seconds            
		ptxt_model_size=0.00 MB            
		ctxt_model_size=0.00 MB            
		rtime_total=0.39 seconds
	- NODE_c1d19c8a-47c3-4ead-b2d0-5ec0

In [15]:
def test_accuracy(net, test_loader):
    """Test the network: measure accuracy on the test set."""

    # Freeze normalization layers
    net.eval()

    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the batches
    idx = 0
    for data, target in test_loader:
        # Accumulate the ground truth labels
        endidx = idx + target.shape[0]
        all_targets[idx:endidx] = target.numpy()

        # Run forward and get the predicted class id
        logits = torch.sigmoid(net(data))
        output = logits.argmax(1).detach().numpy()
        all_y_pred[idx:endidx] = output

        idx += target.shape[0]

    # Print out the accuracy as a percentage
    n_correct = np.sum(all_targets == all_y_pred)
    print(
        f"Test accuracy over plaintext model: "
        f"{n_correct / len(test_loader) * 100:.2f}%"
    )

test_dataset = FedHeartDisease(center=0,pooled=True, train=False, data_path=DATASET_TEST_PATH)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [16]:
def evaluate_model(exp, num_rounds, test_dataloader):
    for r in range(num_rounds):
        fed_model = exp.training_plan().model()
        fed_model.load_state_dict(exp.aggregated_params()[r]['params'])
        test_accuracy(fed_model, test_dataloader)



In [17]:
evaluate_model(exp_no_sec_agg, num_rounds, test_dataloader)


Test accuracy over plaintext model: 48.82%
Test accuracy over plaintext model: 49.21%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 50.00%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 51.57%
Test accuracy over plaintext model: 51.97%
Test accuracy over plaintext model: 53.15%
Test accuracy over plaintext model: 53.54%
Test accuracy over plaintext model: 53.15%
Test accuracy over plaintext model: 54.33%
Test accuracy over plaintext model: 57.09%
Test accuracy over plaintext model: 57.87%
Test accuracy over plaintext model: 59.06%
Test accuracy over plaintext model: 60.24%
Test accuracy over plaintext model: 62.60%
Test accuracy over plaintext model: 63.78%
Test accuracy over plaintext model: 64.17%
Test accuracy over plaintext model: 65.35%
Test accuracy over plaintext model: 65.75%
Test accuracy over plaintext model: 66.14%
Test accura

In [18]:

exp_sec_agg_lom = Experiment(tags=tags,
                 training_plan_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 model_args=model_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                 secagg=SecureAggregation(active=True, scheme='flamingo'))
exp_sec_agg_lom.set_retain_full_history(True)

2024-05-25 09:06:50,352 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2024-05-25 09:06:50,363 fedbiomed DEBUG - Node: NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 polling for the tasks

2024-05-25 09:06:50,365 fedbiomed DEBUG - Node: NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 polling for the tasks

2024-05-25 09:06:50,367 fedbiomed DEBUG - Node: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 polling for the tasks

2024-05-25 09:06:50,369 fedbiomed DEBUG - Node: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 polling for the tasks

...

2024-05-25 09:06:50,374 fedbiomed INFO - Node selected for training -> NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30

2024-05-25 09:06:50,375 fedbiomed INFO - Node selected for training -> NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8

2024-05-25 09:06:50,378 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0043/model_dcd8717e-4768-41e0-8b64-ff2053575a12.py

Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.


True

In [19]:
exp_sec_agg_lom.run()

2024-05-25 09:06:54,686 fedbiomed INFO - Sampled nodes in round 0 ['NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4', 'NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42', 'NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30', 'NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8']

2024-05-25 09:06:54,688 fedbiomed INFO - Sending request 
					 To: NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:06:54,689 fedbiomed INFO - Sending request 
					 To: NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:06:54,690 fedbiomed INFO - Sending request 
					 To: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-05-25 09:06:54,691 fedbiomed INFO - Sending request 
					 To: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 
					 Request: : TRAIN
 -----------------------------------------------------------------

...

2024-05-25 09:07:34,598 fedbiomed INFO - Nodes that successfully reply in round 74 ['NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4', 'NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42', 'NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30', 'NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8']

2024-05-25 09:07:34,599 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.

2024-05-25 09:07:34,602 fedbiomed INFO - Aggregating parameters from 4 nodes.

2024-05-25 09:07:34,603 fedbiomed DEBUG - Aggregation is completed in 0.0 seconds.

75

In [20]:
print_training_rounds(exp_sec_agg_lom, num_rounds)


List the training rounds: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74])

List the nodes for the last training round and their timings:
	- NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 :            
		rtime_training=0.23 seconds            
		ptime_training=0.33 seconds            
		time_encrypt=0.00 seconds            
		ptxt_model_size=0.01 MB            
		ctxt_model_size=0.01 MB            
		rtime_total=0.50 seconds
	- NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 :            
		rtime_training=0.18 seconds            
		ptime_training=0.30 seconds            
		time_encrypt=0.00 seconds            
		ptxt_model_size=0.01 MB            
		ctxt_model_size=0.01 MB            
		rtime_total=0.50 seconds
	- NODE_c1d19c8a-47c3-4ead-b2d0-5ec0

In [21]:
evaluate_model(exp_sec_agg_lom, num_rounds, test_dataloader)

Test accuracy over plaintext model: 48.82%
Test accuracy over plaintext model: 49.21%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 50.00%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 51.57%
Test accuracy over plaintext model: 51.97%
Test accuracy over plaintext model: 52.76%
Test accuracy over plaintext model: 53.54%
Test accuracy over plaintext model: 53.15%
Test accuracy over plaintext model: 54.33%
Test accuracy over plaintext model: 56.69%
Test accuracy over plaintext model: 57.87%
Test accuracy over plaintext model: 58.66%
Test accuracy over plaintext model: 60.24%
Test accuracy over plaintext model: 61.81%
Test accuracy over plaintext model: 63.39%
Test accuracy over plaintext model: 64.17%
Test accuracy over plaintext model: 65.35%
Test accuracy over plaintext model: 65.75%
Test accuracy over plaintext model: 66.14%
Test accura

2024-05-25 09:16:57,373 fedbiomed DEBUG - Node: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 polling for the tasks

2024-05-25 09:17:04,534 fedbiomed DEBUG - Node: NODE_1ab3470a-b961-4de6-a454-237155e67b4c polling for the tasks

2024-05-25 09:17:53,819 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

2024-05-25 09:19:52,786 fedbiomed DEBUG - Node: NODE_7849b47e-12fc-4062-a85a-706aa967e395 polling for the tasks

2024-05-25 09:19:55,595 fedbiomed DEBUG - Node: NODE_1ab3470a-b961-4de6-a454-237155e67b4c polling for the tasks

2024-05-25 09:19:58,877 fedbiomed DEBUG - Node: NODE_efae9061-c6e3-4c90-87eb-ba8775edd565 polling for the tasks

In [1]:
exp_sec_agg_jls = Experiment(tags=tags,
                 training_plan_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 model_args=model_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                 secagg=SecureAggregation(active=True, scheme='jls'), # or custom SecureAggregation(active=<bool>, clipping_range=<int>)
                 tensorboard=True
                )
exp_sec_agg_jls.set_retain_full_history(True)

NameError: name 'Experiment' is not defined

In [19]:
tensorboard --logdir "$tensorboard_dir"

Reusing TensorBoard on port 6006 (pid 45336), started 0:08:17 ago. (Use '!kill 45336' to kill it.)

In [20]:
exp_sec_agg_jls.run()

2024-05-24 18:12:24,006 fedbiomed DEBUG - Secagg context for default_biprime0 is already existing on researcher researcher_id='RESEARCHER_596f170d-96d8-417a-8ddf-9bdb1cf7d52b'

2024-05-24 18:12:24,010 fedbiomed DEBUG - Node: NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30 polling for the tasks

2024-05-24 18:12:24,025 fedbiomed DEBUG - Node: NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 polling for the tasks

2024-05-24 18:12:24,026 fedbiomed DEBUG - Node: NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 polling for the tasks

2024-05-24 18:12:24,029 fedbiomed DEBUG - Node: NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 polling for the tasks

...

2024-05-24 18:13:05,030 fedbiomed INFO - Nodes that successfully reply in round 49 ['NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4', 'NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42', 'NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8', 'NODE_c1d19c8a-47c3-4ead-b2d0-5ec000f29a30']

2024-05-24 18:13:05,032 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.

2024-05-24 18:13:05,073 fedbiomed INFO - Aggregating 4 parameters from 4 nodes.

2024-05-24 18:13:05,075 fedbiomed DEBUG - Aggregation is completed in 0.04 seconds.

50

In [21]:
print("\nList the training rounds : ", exp_sec_agg_jls.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
for rnd in range(num_rounds):
    round_data = exp_sec_agg_jls.training_replies()[rnd]
    for r in round_data.values():
        print("\t- {id} :\
        \n\t\trtime_training={rtraining:.2f} seconds\
        \n\t\tptime_training={ptraining:.2f} seconds\
        \n\t\ttime_encrypt={time_encrypt:.2f} seconds\
        \n\t\tptxt_model_size={ptxt_model_size:.2f} MB\
        \n\t\tctxt_model_size={ctxt_model_size:.2f} MB\
        \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
            rtraining = r['timing']['rtime_training'],
            ptraining = r['timing']['ptime_training'],
            time_encrypt = r['timing']['time_encrypt'],
            ptxt_model_size = r['communication']['ptxt_model_size'],
            ctxt_model_size = r['communication']['ctxt_model_size'],
            rtotal = r['timing']['rtime_total'],))
    print('\n')


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

List the nodes for the last training round and their timings : 
	- NODE_86fdd95e-cb89-4f45-8da8-99a31f903ef4 :        
		rtime_training=0.08 seconds        
		ptime_training=0.17 seconds        
		time_encrypt=0.04 seconds        
		ptxt_model_size=0.01 MB        
		ctxt_model_size=0.00 MB        
		rtime_total=0.50 seconds
	- NODE_0fcf7cdb-9c90-4e2c-8362-016ccc5ebe42 :        
		rtime_training=0.17 seconds        
		ptime_training=0.29 seconds        
		time_encrypt=0.09 seconds        
		ptxt_model_size=0.01 MB        
		ctxt_model_size=0.00 MB        
		rtime_total=0.50 seconds
	- NODE_46d649b9-17bb-41be-9f7f-acdb5fc6dfa8 :        
		rtime_training=0.23 seconds        
		ptime_training=0.31 seconds        
		time_encrypt=0.09 seconds        
		ptxt_model_

In [37]:
for r in range(num_rounds):
    fed_model = exp_sec_agg_jls.training_plan().model()
    fed_model.load_state_dict(exp_sec_agg_jls.aggregated_params()[r]['params'])
    test_accuracy(fed_model,test_dataloader)

Test accuracy over plaintext model: 48.82%
Test accuracy over plaintext model: 49.21%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 49.61%
Test accuracy over plaintext model: 50.00%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 50.39%
Test accuracy over plaintext model: 51.57%
Test accuracy over plaintext model: 51.97%
Test accuracy over plaintext model: 52.76%
Test accuracy over plaintext model: 53.54%
Test accuracy over plaintext model: 53.15%
Test accuracy over plaintext model: 54.33%
Test accuracy over plaintext model: 56.69%
Test accuracy over plaintext model: 57.87%
Test accuracy over plaintext model: 58.66%
Test accuracy over plaintext model: 60.24%
Test accuracy over plaintext model: 61.81%
Test accuracy over plaintext model: 63.39%
Test accuracy over plaintext model: 64.17%
Test accuracy over plaintext model: 65.35%
Test accuracy over plaintext model: 65.75%
Test accuracy over plaintext model: 66.14%
Test accura